In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Importing the route data

In [17]:
# Get Directory
BASE_DIR = "D:/Code/RoutingChallenge/routing-challenge-az"

training_package=os.path.join(BASE_DIR, 'data/model_build_inputs/package_data.json')

# Read route data
print('Reading Input Data')
df_pkg = pd.read_json(training_routes_path).T

Reading Input Data


In [18]:
df_pkg.shape

(6112, 675)

In [19]:
675-df_pkg.head().isnull().sum(axis=1)

RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77    119
RouteID_0016bc70-cb8d-48b0-aa55-8ee50bdcdb59    106
RouteID_001948e9-4675-486d-9ec5-912fd8e0770f    128
RouteID_001b4ee3-c4f2-467f-932b-c85524d1021f    142
RouteID_0021a2aa-780f-460d-b09a-f301709e2523    155
dtype: int64

It matches with number of stops

In [31]:
pkgs = df_pkg.iloc[0,:].dropna().iloc[10]
pkgs

{'PackageID_8a90dea1-cf2c-40f8-b91c-4998d32c4b9d': {'scan_status': 'DELIVERED',
  'time_window': {'start_time_utc': '2018-07-27 16:00:00',
   'end_time_utc': '2018-07-27 23:00:00'},
  'planned_service_time_seconds': 47.5,
  'dimensions': {'depth_cm': 25.4, 'height_cm': 12.7, 'width_cm': 17.8}},
 'PackageID_d1d64da8-ade4-4936-9450-b28af876f75a': {'scan_status': 'DELIVERED',
  'time_window': {'start_time_utc': '2018-07-27 16:00:00',
   'end_time_utc': '2018-07-27 23:00:00'},
  'planned_service_time_seconds': 47.5,
  'dimensions': {'depth_cm': 33.0, 'height_cm': 20.3, 'width_cm': 26.7}}}

In [187]:
def get_time_windows_package(pkgs, handle_mismatch='intersection',verbose=False):
    time_windows = {}
    start,end = None,None
#     print(pkgs)
    if pkgs is np.nan:
        return None
    for idd,pkg in pkgs.items():
        if verbose:
            print(start,end)
        time_window = pkg['time_window']
        if verbose: 
            print('>>',end='   ')
            print(time_window['start_time_utc'],end='   ')
            print(time_window['end_time_utc'])
        if start is None and end is None:
            start = time_window['start_time_utc']
            end = time_window['end_time_utc']
            if start is not None:
                start = np.datetime64(start)
            if end is not None:
                end = np.datetime64(end)
        else:
            new_start = time_window['start_time_utc']
            new_end = time_window['end_time_utc']
            
            if new_start is not None:
                new_start = np.datetime64(new_start)
            if new_end is not None:
                new_end = np.datetime64(new_end)
                
            if new_start != start and new_start is not None:
                if new_start > end:
                    print(f"Start times mismatch for {idd}")
                else:
                    start = min(start, new_start)
            if new_end != end and new_end is not None:
                if new_end < start:
                    print(f"End times mismatch for {idd}")
                else:
                    end = max(end, new_end)
                
#     print(start,end)
#     print(type(start))
    return start,end

In [188]:
pkgs = df_pkg.iloc[0,0]
get_time_windows_package(pkgs)

(numpy.datetime64('2018-07-27T16:00:00'),
 numpy.datetime64('2018-07-28T00:00:00'))

In [204]:
def get_time_windows_all(df_pkg):
    df_tw = df_pkg.copy()
    for route in df_pkg.index:
#         print(route)
#         print(df_pkg.loc[route,:])
        df_tw.loc[route,:] = df_pkg.loc[route,:].apply(lambda x: get_time_windows_package(x))
    return df_tw

In [205]:
df_tw = get_time_windows_all(df_pkg)

In [206]:
df_tw.shape

(6112, 675)

In [209]:
len(df_tw.iloc[0,:].dropna())

119